# Feat 1 Check  
Here we go over quality checking our feat1 analysis directories generated by FSL!  
   
Reference: [Checking your level 1 - mumfordbrainstats](https://www.youtube.com/watch?v=kW1xgGs4NUU&list=PLB2iAtgpI4YHlH4sno3i3CUjCofI38a-3&index=13)  
#### Checklist:  
* Create html  
* Check for exsistence of expected files  
    * zstats   
    * look into log files (errors, etc...)  



#### When looking at html: 
* Regressors look like you expect 
* check pairwise colinearity

In [1]:
import os, glob
import subprocess
from subprocess import check_output
import pandas as pd
pd.set_option('display.max_rows', 159)
pd.options.display.max_rows

159

In [2]:
#python feat1_fsf_generator+.py -deriv_dir /projects/niblab/bids_projects/Experiments/bbx/bids/derivatives \
#-fsf /projects/niblab/bids_projects/Experiments/bbx/bids/code/design.fsf \
#-all_runs -sess 1 -task training -evs SSBcue USBcue H2Ocue SSBtaste USBtaste H2Otaste rinse

In [3]:
# get feat1 directories
deriv_path="/projects/niblab/bids_projects/Experiments/bbx/bids/derivatives"#input("Enter target study bids path: ")
#/projects/niblab/bids_projects/Experiments/bbx/bids

### Get subjects  and set variables

In [4]:
subjects = sorted(glob.glob(os.path.join(deriv_path, 'sub-*/ses-1')))
omit = ['sub-074', 'sub-157']
sub_ids = [x.split("/")[-2] for x in subjects if x.split("/")[-2] not in omit]
print(len(sub_ids))

129


In [5]:
sub_ct=len(subjects)
# 25% of subject ct variable-
percent = int(sub_ct*.25)
print("Subject count: {}\t25%:{}".format(sub_ct,percent))


Subject count: 129	25%:32


### Write subject report

In [6]:
outfile = os.path.join(deriv_path, "testing_feat1_qc.html")
f = open(outfile, "w")

print("> Writing QC report for {} subjects, 25% of the subjects.".format(percent))

for sub_path in subjects[:percent]:
    sub_id = sub_path.split("/")[-2]
    f.write("<h2>Subject: %s\n</h2>"%sub_id)
    feat1_path = os.path.join(sub_path, "func/Analysis/feat1")
    feat1_dirs = sorted(glob.glob(os.path.join(feat1_path, "*.feat")))
    for f1_dir in feat1_dirs:
        run_id = f1_dir.split("/")[-1].split("_")[1]
        f.write("<h4>Directory: %s\n</h4>"%f1_dir)

        design_img1 = "%s/design.png"%f1_dir
        if os.path.exists(design_img1):
            f.write("<p><img src=%s alt=img1"%(design_img1))
            f.write("\n")
        design_img2 = "%s/design_cov.png"%f1_dir
        if os.path.exists(design_img2):
            f.write("<p><IMG SRC=%s alt=img2"%(design_img2))
            f.write("\n")
        func2high_img = "%s/reg/example_func2highres.png"%f1_dir
        if os.path.exists(func2high_img):
            f.write("<p><IMG SRC=%s WIDTH=1200 alt=img3>"%(func2high_img))
            f.write("\n")
        func2std_img = "%s/reg/example_func2standard.png"%f1_dir
        if os.path.exists(func2std_img ):
            f.write("<p><IMG SRC=%s WIDTH=1200 alt=img4>"%(func2std_img))
            f.write("\n")
        high2std_img = "%s/reg/highres2standard.png"%f1_dir
        if os.path.exists(high2std_img ):
            f.write("<p><IMG SRC=%s WIDTH=1200 alt=img5>"%(high2std_img))
            f.write("\n")
        
        
        
f.close()

print("> Completed writing report to file {}".format(outfile))

> Writing QC report for 32 subjects, 25% of the subjects.
> Completed writing report to file /projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/testing_feat1_qc.html


### Checking for zstats

In [8]:
print("Subjects with missing zstats: \n")

for sub_path in subjects:
    feat1_path = os.path.join(sub_path, "func/Analysis/feat1")
    feat1_dirs = sorted(glob.glob(os.path.join(feat1_path, "*.feat")))
    for run_path in feat1_dirs:
        #print(run_path)
        run_id = run_path.split("/")[-1].split("_")[1].split(".")[0]
        zstats=glob.glob(os.path.join(run_path, 'stats/zstat*nii.gz'))
        if not zstats:
            print("subject: {} \trun: {}".format(sub_path.split("/")[-2], run_id))
print("\n> Completed.")

Subjects with missing zstats: 


> Completed.


## Check Activation

In [9]:
qc_dict={}

In [10]:
mean_func_ct=0
for sub_dir in subjects:
    #print(sub_dir)
    sub_id=sub_dir.split("/")[-2]
    if sub_id not in qc_dict:
        qc_dict[sub_id] = {}
        
    feat1_folders=glob.glob(os.path.join(sub_dir, 'func/Analysis/feat1/training*.feat'))
    #print(feat1_folders)
    for run_folder in feat1_folders:
        
        run_id = str(run_folder.split("/")[-1].split("_")[1].split(".")[0])
        print(run_id)
        if run_id not in qc_dict[sub_id]:
            qc_dict[sub_id][run_id] = None
            print('making dictionary')
        filtered_func = os.path.join(run_folder, "filtered_func_data.nii.gz")
        zstat = os.path.join(run_folder, "stats/zstat1.nii.gz")
        if not os.path.exists(zstat):
            print("no zstat")
        #print("zstat files: {} \n".format(zstat))
        shell_cmd = "cluster -i {} -t 0 > temp_file.txt".format(zstat)
        #print("cluster cmd: {} \n".format(shell_cmd))
        try:
            cluster = check_output(shell_cmd, shell=True)
            temp=pd.read_csv("temp_file.txt", sep='\t')
            max_vox=temp["MAX"].max()
        except:
            max_vox=None
        
        qc_dict[sub_id][run_id]=max_vox



run-1
making dictionary
run-2
making dictionary
run-3
making dictionary
run-4
making dictionary
run-1
making dictionary
run-2
making dictionary
run-3
making dictionary
run-4
making dictionary
run-1
making dictionary
run-2
making dictionary
run-3
making dictionary
run-4
making dictionary
run-1
making dictionary
run-2
making dictionary
run-4
making dictionary
run-1
making dictionary
run-2
making dictionary
run-3
making dictionary
run-4
making dictionary
run-1
making dictionary
run-2
making dictionary
run-3
making dictionary
run-4
making dictionary
run-1
making dictionary
run-2
making dictionary
run-3
making dictionary
run-4
making dictionary
run-1
making dictionary
run-2
making dictionary
run-3
making dictionary
run-4
making dictionary
run-1
making dictionary
run-2
making dictionary
run-3
making dictionary
run-4
making dictionary
run-1
making dictionary
run-2
making dictionary
run-3
making dictionary
run-4
making dictionary
run-1
making dictionary
run-2
making dictionary
run-3
making dic

run-3
making dictionary
run-4
making dictionary
run-1
making dictionary
run-2
making dictionary
run-3
making dictionary
run-4
making dictionary
run-1
making dictionary
run-2
making dictionary
run-3
making dictionary
run-4
making dictionary
run-1
making dictionary
run-2
making dictionary
run-3
making dictionary
run-4
making dictionary
run-1
making dictionary
run-2
making dictionary
run-3
making dictionary
run-4
making dictionary
run-1
making dictionary
run-2
making dictionary
run-3
making dictionary
run-4
making dictionary
run-1
making dictionary
run-2
making dictionary
run-3
making dictionary
run-4
making dictionary
run-1
making dictionary
run-2
making dictionary
run-3
making dictionary
run-4
making dictionary
run-1
making dictionary
run-2
making dictionary
run-3
making dictionary
run-4
making dictionary
run-1
making dictionary
run-2
making dictionary
run-3
making dictionary
run-4
making dictionary
run-1
making dictionary
run-2
making dictionary
run-3
making dictionary
run-4
making dic

In [11]:
qc_df = pd.DataFrame(qc_dict).T


In [12]:
qc_df

,run-1,run-2,run-3,run-4
sub-001,3.26,4.11,5.10,4.89
sub-002,4.02,4.16,4.88,3.66
sub-003,5.10,5.24,3.90,4.05
sub-004,5.05,4.50,NaN,3.44
sub-005,8.44,5.00,5.11,4.47
sub-006,4.65,4.07,4.39,3.83
sub-007,3.50,3.68,3.97,4.40
sub-008,6.30,6.19,4.45,4.88
sub-009,5.27,4.06,4.14,4.50
sub-010,4.14,4.21,4.91,5.18


In [13]:
error_subs = qc_df[qc_df.isna().any(axis=1)]


In [14]:
sub_3_runs=[ 'sub-048', 'sub-070', 'sub-076', 'sub-146', 'sub-147', 'sub-004', 'sub-070', 'sub-079']
sub_2_runs=['sub-060']



In [15]:
for sub in error_subs.index.values:
    if sub in sub_3_runs or sub in sub_2_runs:
        error_subs.drop(sub, inplace=True)

/home/nbytes/modules/software/miniconda3/3.0.0/lib/python3.7/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [16]:
error_subs

,run-1,run-2,run-3,run-4


In [17]:
error_subs.index.values

array([], dtype=object)

### Checking for errors in logs

In [11]:
for sub_path in subjects:
    feat1_path = os.path.join(sub_path, "func/Analysis/feat1")
    feat1_dirs = sorted(glob.glob(os.path.join(feat1_path, "*.feat")))
    #for run_dir in feat1_dirs:
        #html = os.path.join(run_dir, "report_log.html")
        #cmd = ["grep", "-i", "'error'", "%s"%html]
        #print(subprocess.call(cmd,  stdout=subprocess.PIPE, shell=True))
        #output, err = cmd.communicate()
        #if not output:
         #   pass
        #else:
         #   print(output)